In [39]:
!pip install bayesian-optimization

In [40]:
# ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from bayes_opt import BayesianOptimization
import xgboost as xgb

In [41]:
df = pd.read_csv("Microbiota_composition.csv")
df = df.drop("Unnamed: 0", axis=1)

In [ ]:
# Features
X = df.drop(columns=["condition"], axis=1)

# Target
y = df["condition"]

# lambda to scale the target into numeric (CRC : 1, control : 0)
y = y.apply(lambda x: 1 if x == "CRC" else 0)

# Split df into train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Boolean filter to keep numeric column
numeric_features = df.select_dtypes(include=['number']).columns
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# Selection of the categorical feature to preprocess
# categorical_feature = X.select_dtypes(exclude=['number']).columns
# feature_transformer = Pipeline(steps=[("encoder", OneHotEncoder(drop="first"))])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        # ("cat", feature_transformer, categorical_feature),
    ]
)

# Prétraitement des données (X_train, X_test) avec une normalisation
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
# input_shape = X_train_scaled.shape[1]

# Définir la fonction d'évaluation
def evaluate_xgb(n_estimators, max_depth, learning_rate, gamma, min_child_weight):
    model = xgb.XGBClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        gamma=gamma,
        objective='binary:logistic',
        min_child_weight=min_child_weight,
        random_state=42
        )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Définir l'espace de recherche
pbounds = {
    'n_estimators': (50, 250),
    'max_depth': (3, 6),
    'learning_rate': (0.01, 0.1),
    'gamma': (0, 0.4),
    'min_child_weight': (1, 2)
    }

# Initialiser l'optimiseur bayésien
optimizer = BayesianOptimization(f=evaluate_xgb, pbounds=pbounds, random_state=42)

# Lancer l'optimisation
optimizer.maximize(init_points=5, n_iter=50)

# Récupérer les hyperparamètres optimaux
best_params = optimizer.max['params']

# Entraîner le modèle final avec les meilleurs hyperparamètres
best_model = xgb.XGBClassifier(n_estimators=int(best_params['n_estimators']),
                           max_depth=int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'],
                           gamma=best_params['gamma'],
                           min_child_weight=best_params['min_child_weight'])
best_model.fit(X_train, y_train)

|   iter    |  target   |   gamma   | learni... | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------------------
| 1         | 0.7987    | 0.1498    | 0.09556   | 5.928     | 1.599     | 81.2      |
| 2         | 0.7727    | 0.0624    | 0.01523   | 6.465     | 1.601     | 191.6     |
| 3         | 0.7922    | 0.008234  | 0.09729   | 6.33      | 1.212     | 86.36     |
| 4         | 0.7662    | 0.07336   | 0.03738   | 5.099     | 1.432     | 108.2     |
| 5         | 0.8052    | 0.2447    | 0.02255   | 4.169     | 1.366     | 141.2     |
| 6         | 0.7792    | 0.341     | 0.1       | 3.0       | 1.979     | 154.9     |
| 7         | 0.7468    | 0.4       | 0.1       | 7.0       | 2.0       | 134.2     |
| 8         | 0.7727    | 0.03547   | 0.02595   | 3.709     | 1.536     | 143.6     |
| 9         | 0.8182    | 0.1874    | 0.09937   | 6.687     | 1.432     | 113.1     |
| 10        | 0.8052    | 0.323     | 0.02582   | 4.26

In [ ]:
# Make predictions on the test set
predictions = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.4f}")